In [15]:
import tensorflow as tf
import pandas as pd
from pathlib import Path
from sklearn import tree
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

import os
from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
# Load data
file_path = Path("Resources/final_sample.csv")
df = pd.read_csv(file_path)
df.head()

,agency_code,loan_type,property_type,loan_purpose,owner_occupancy,loan_amount_000s,preapproval,action_taken,applicant_ethnicity,co_applicant_ethnicity,...,purchaser_type,hoepa_status,lien_status,population,minority_population,hud_median_family_income,tract_to_msamd_income,number_of_owner_occupied_units,number_of_1_to_4_family_units,Region
0,CFPB,Conventional,One to Four-Family,Refinancing,Owner Occupied,320.0,Not Applicable,Application Approved but not Accepted,Not Hispanic or Latino,No Co-Applicant,...,Loan Not Originated,Not a HOEPA Loan,Secured by First Lien,4987.0,6.66,79600.0,119.47,1621.0,1932.0,MW
1,HUD,Conventional,One to Four-Family,Refinancing,Not Owner Occupied,305.0,Not Applicable,Application Approved but not Accepted,Not Hispanic or Latino,No Co-Applicant,...,Loan Not Originated,Not a HOEPA Loan,Secured by First Lien,2569.0,90.31,72500.0,66.44,424.0,1249.0,SW
2,FDIC,Conventional,One to Four-Family,Home Purchase,Not Owner Occupied,310.0,Preapproval Not Requested,Application Approved but not Accepted,Not Hispanic or Latino,Not Hispanic or Latino,...,Loan Not Originated,Not a HOEPA Loan,Secured by First Lien,2965.0,5.63,63200.0,273.83,1424.0,3916.0,W
3,CFPB,Conventional,One to Four-Family,Home Purchase,Owner Occupied,507.0,Not Applicable,Application Approved but not Accepted,Not Hispanic or Latino,Not Hispanic or Latino,...,Loan Not Originated,Not a HOEPA Loan,Secured by First Lien,6097.0,11.94,70900.0,233.93,2095.0,2135.0,SW
4,HUD,VA-Guaranteed,One to Four-Family,Refinancing,Owner Occupied,377.0,Not Applicable,Application Approved but not Accepted,Not Hispanic or Latino,No Co-Applicant,...,Loan Not Originated,Not a HOEPA Loan,Secured by First Lien,4418.0,12.02,75200.0,115.37,1217.0,1628.0,W


In [4]:
# value counts of agency code column
df["action_taken"].value_counts()

Loan Originated                                        68972
Application Denied by Financial Institution            18552
Application Withdrawn by Applicant                     15496
Loan Purchased by the Institution                       9488
File Closed for Incompleteness                          4998
Application Approved but not Accepted                   3706
Preapproval Request Denied by Financial Institution        5
Preapproval Request Approved but not Accepted              2
Name: action_taken, dtype: int64

In [5]:
# convert action taken column to approved/denied
action = {
    'Loan Originated': 1, 
    'Loan Purchased by the Institution': 1, 
    'Application Denied by Financial Institution': 0,
    'Application Withdrawn by Applicant': 2,
    'File Closed for Incompleteness': 2,
    'Application Approved but not Accepted': 0,
    'Preapproval Request Denied by Financial Institution': 0,
    'Preapproval Request Approved but not Accepted': 0}

df["action_taken_summary"] = df["action_taken"].map(action)

In [6]:
df.count()

agency_code                       121219
loan_type                         121219
property_type                     121219
loan_purpose                      121219
owner_occupancy                   121219
loan_amount_000s                  121219
preapproval                       121219
action_taken                      121219
applicant_ethnicity               121219
co_applicant_ethnicity            121219
applicant_race_1                  121219
co_applicant_race_1               121219
applicant_sex                     121219
co_applicant_sex                  121219
applicant_income_000s             121219
purchaser_type                    121219
hoepa_status                      121219
lien_status                       121219
population                        121219
minority_population               121219
hud_median_family_income          121219
tract_to_msamd_income             121219
number_of_owner_occupied_units    121219
number_of_1_to_4_family_units     121219
Region          

In [7]:
# value counts of action taken summary column
df["action_taken_summary"].value_counts()

1    78460
0    22265
2    20494
Name: action_taken_summary, dtype: int64

In [8]:
# drop rows with the value 2 in the action taken summary
df = df[df.action_taken_summary != 2]

In [9]:
# check dataframe
df.head(10)

,agency_code,loan_type,property_type,loan_purpose,owner_occupancy,loan_amount_000s,preapproval,action_taken,applicant_ethnicity,co_applicant_ethnicity,...,hoepa_status,lien_status,population,minority_population,hud_median_family_income,tract_to_msamd_income,number_of_owner_occupied_units,number_of_1_to_4_family_units,Region,action_taken_summary
0,CFPB,Conventional,One to Four-Family,Refinancing,Owner Occupied,320.0,Not Applicable,Application Approved but not Accepted,Not Hispanic or Latino,No Co-Applicant,...,Not a HOEPA Loan,Secured by First Lien,4987.0,6.66,79600.0,119.47,1621.0,1932.0,MW,0
1,HUD,Conventional,One to Four-Family,Refinancing,Not Owner Occupied,305.0,Not Applicable,Application Approved but not Accepted,Not Hispanic or Latino,No Co-Applicant,...,Not a HOEPA Loan,Secured by First Lien,2569.0,90.31,72500.0,66.44,424.0,1249.0,SW,0
2,FDIC,Conventional,One to Four-Family,Home Purchase,Not Owner Occupied,310.0,Preapproval Not Requested,Application Approved but not Accepted,Not Hispanic or Latino,Not Hispanic or Latino,...,Not a HOEPA Loan,Secured by First Lien,2965.0,5.63,63200.0,273.83,1424.0,3916.0,W,0
3,CFPB,Conventional,One to Four-Family,Home Purchase,Owner Occupied,507.0,Not Applicable,Application Approved but not Accepted,Not Hispanic or Latino,Not Hispanic or Latino,...,Not a HOEPA Loan,Secured by First Lien,6097.0,11.94,70900.0,233.93,2095.0,2135.0,SW,0
4,HUD,VA-Guaranteed,One to Four-Family,Refinancing,Owner Occupied,377.0,Not Applicable,Application Approved but not Accepted,Not Hispanic or Latino,No Co-Applicant,...,Not a HOEPA Loan,Secured by First Lien,4418.0,12.02,75200.0,115.37,1217.0,1628.0,W,0
5,FDIC,FHA-Insured,One to Four-Family,Refinancing,Owner Occupied,206.0,Not Applicable,Application Approved but not Accepted,Not Hispanic or Latino,No Co-Applicant,...,Not a HOEPA Loan,Secured by First Lien,5568.0,36.66,64800.0,102.41,1861.0,2495.0,SE,0
6,CFPB,Conventional,One to Four-Family,Refinancing,Owner Occupied,150.0,Not Applicable,Application Approved but not Accepted,Not Hispanic or Latino,No Co-Applicant,...,Not a HOEPA Loan,Secured by First Lien,2236.0,6.84,66200.0,80.47,1057.0,1808.0,SW,0
7,OCC,Conventional,One to Four-Family,Home Improvement,Owner Occupied,39.0,Not Applicable,Application Approved but not Accepted,Not Hispanic or Latino,Not Hispanic or Latino,...,Not a HOEPA Loan,Secured by a Subordinate Loan,8119.0,19.53,72500.0,130.01,1961.0,2443.0,SW,0
8,NCUA,Conventional,One to Four-Family,Home Purchase,Owner Occupied,223.0,Preapproval Requested,Application Approved but not Accepted,Not Hispanic or Latino,Not Hispanic or Latino,...,Not a HOEPA Loan,Secured by First Lien,4791.0,44.29,73900.0,81.86,975.0,1429.0,MW,0
9,CFPB,Conventional,One to Four-Family,Refinancing,Owner Occupied,61.0,Not Applicable,Application Approved but not Accepted,Not Hispanic or Latino,No Co-Applicant,...,Not a HOEPA Loan,Secured by First Lien,11138.0,38.91,57900.0,81.92,1880.0,3885.0,SE,0


In [10]:
# check counts
df.count()

agency_code                       100725
loan_type                         100725
property_type                     100725
loan_purpose                      100725
owner_occupancy                   100725
loan_amount_000s                  100725
preapproval                       100725
action_taken                      100725
applicant_ethnicity               100725
co_applicant_ethnicity            100725
applicant_race_1                  100725
co_applicant_race_1               100725
applicant_sex                     100725
co_applicant_sex                  100725
applicant_income_000s             100725
purchaser_type                    100725
hoepa_status                      100725
lien_status                       100725
population                        100725
minority_population               100725
hud_median_family_income          100725
tract_to_msamd_income             100725
number_of_owner_occupied_units    100725
number_of_1_to_4_family_units     100725
Region          

In [ ]:
# drop action taken column
df = df.drop(columns=["action_taken"])

In [14]:
# check df again
df.head(10)

,agency_code,loan_type,property_type,loan_purpose,owner_occupancy,loan_amount_000s,preapproval,applicant_ethnicity,co_applicant_ethnicity,applicant_race_1,...,hoepa_status,lien_status,population,minority_population,hud_median_family_income,tract_to_msamd_income,number_of_owner_occupied_units,number_of_1_to_4_family_units,Region,action_taken_summary
0,CFPB,Conventional,One to Four-Family,Refinancing,Owner Occupied,320.0,Not Applicable,Not Hispanic or Latino,No Co-Applicant,White,...,Not a HOEPA Loan,Secured by First Lien,4987.0,6.66,79600.0,119.47,1621.0,1932.0,MW,0
1,HUD,Conventional,One to Four-Family,Refinancing,Not Owner Occupied,305.0,Not Applicable,Not Hispanic or Latino,No Co-Applicant,White,...,Not a HOEPA Loan,Secured by First Lien,2569.0,90.31,72500.0,66.44,424.0,1249.0,SW,0
2,FDIC,Conventional,One to Four-Family,Home Purchase,Not Owner Occupied,310.0,Preapproval Not Requested,Not Hispanic or Latino,Not Hispanic or Latino,White,...,Not a HOEPA Loan,Secured by First Lien,2965.0,5.63,63200.0,273.83,1424.0,3916.0,W,0
3,CFPB,Conventional,One to Four-Family,Home Purchase,Owner Occupied,507.0,Not Applicable,Not Hispanic or Latino,Not Hispanic or Latino,Asian,...,Not a HOEPA Loan,Secured by First Lien,6097.0,11.94,70900.0,233.93,2095.0,2135.0,SW,0
4,HUD,VA-Guaranteed,One to Four-Family,Refinancing,Owner Occupied,377.0,Not Applicable,Not Hispanic or Latino,No Co-Applicant,White,...,Not a HOEPA Loan,Secured by First Lien,4418.0,12.02,75200.0,115.37,1217.0,1628.0,W,0
5,FDIC,FHA-Insured,One to Four-Family,Refinancing,Owner Occupied,206.0,Not Applicable,Not Hispanic or Latino,No Co-Applicant,Black or African American,...,Not a HOEPA Loan,Secured by First Lien,5568.0,36.66,64800.0,102.41,1861.0,2495.0,SE,0
6,CFPB,Conventional,One to Four-Family,Refinancing,Owner Occupied,150.0,Not Applicable,Not Hispanic or Latino,No Co-Applicant,White,...,Not a HOEPA Loan,Secured by First Lien,2236.0,6.84,66200.0,80.47,1057.0,1808.0,SW,0
7,OCC,Conventional,One to Four-Family,Home Improvement,Owner Occupied,39.0,Not Applicable,Not Hispanic or Latino,Not Hispanic or Latino,White,...,Not a HOEPA Loan,Secured by a Subordinate Loan,8119.0,19.53,72500.0,130.01,1961.0,2443.0,SW,0
8,NCUA,Conventional,One to Four-Family,Home Purchase,Owner Occupied,223.0,Preapproval Requested,Not Hispanic or Latino,Not Hispanic or Latino,White,...,Not a HOEPA Loan,Secured by First Lien,4791.0,44.29,73900.0,81.86,975.0,1429.0,MW,0
9,CFPB,Conventional,One to Four-Family,Refinancing,Owner Occupied,61.0,Not Applicable,Not Hispanic or Latino,No Co-Applicant,White,...,Not a HOEPA Loan,Secured by First Lien,11138.0,38.91,57900.0,81.92,1880.0,3885.0,SE,0


Run Neural Network

In [16]:
# Generate our categorical variable lists
action_cat = df.dtypes[df.dtypes == "object"].index.tolist()
df[action_cat].nunique()

agency_code                6
loan_type                  4
property_type              2
loan_purpose               3
owner_occupancy            3
preapproval                3
applicant_ethnicity        4
co_applicant_ethnicity     5
applicant_race_1           7
co_applicant_race_1        8
applicant_sex              4
co_applicant_sex           5
purchaser_type            10
hoepa_status               2
lien_status                4
Region                     7
dtype: int64

In [17]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(df[action_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(action_cat)
encode_df.head()

,agency_code_CFPB,agency_code_FDIC,agency_code_FRS,agency_code_HUD,agency_code_NCUA,agency_code_OCC,loan_type_Conventional,loan_type_FHA-Insured,loan_type_FSA/RHS,loan_type_VA-Guaranteed,...,lien_status_Not Secured by Lien,lien_status_Secured by First Lien,lien_status_Secured by a Subordinate Loan,Region_MA,Region_MW,Region_NE,Region_NW,Region_SE,Region_SW,Region_W
0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [18]:
# Merge one-hot encoded features and drop the originals
df = df.merge(encode_df,left_index=True, right_index=True)
df = df.drop(action_cat,1)
df.head()

c:\Users\olivi\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,loan_amount_000s,applicant_income_000s,population,minority_population,hud_median_family_income,tract_to_msamd_income,number_of_owner_occupied_units,number_of_1_to_4_family_units,action_taken_summary,agency_code_CFPB,...,lien_status_Not Secured by Lien,lien_status_Secured by First Lien,lien_status_Secured by a Subordinate Loan,Region_MA,Region_MW,Region_NE,Region_NW,Region_SE,Region_SW,Region_W
0,320.0,148.0,4987.0,6.66,79600.0,119.47,1621.0,1932.0,0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,305.0,145.0,2569.0,90.31,72500.0,66.44,424.0,1249.0,0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,310.0,470.0,2965.0,5.63,63200.0,273.83,1424.0,3916.0,0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,507.0,507.0,6097.0,11.94,70900.0,233.93,2095.0,2135.0,0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,377.0,87.0,4418.0,12.02,75200.0,115.37,1217.0,1628.0,0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [19]:
# check values
df.count()

loan_amount_000s            80231
applicant_income_000s       80231
population                  80231
minority_population         80231
hud_median_family_income    80231
                            ...  
Region_NE                   80231
Region_NW                   80231
Region_SE                   80231
Region_SW                   80231
Region_W                    80231
Length: 86, dtype: int64

In [20]:
# Split our preprocessed data into our features and target arrays
y = df["action_taken_summary"].values
X = df.drop(["action_taken_summary"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

c:\Users\olivi\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [21]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [22]:
import keras

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 86
hidden_nodes_layer2 = 40
hidden_nodes_layer3 = 20

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

## Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 86)                7396      
                                                                 
 dense_1 (Dense)             (None, 40)                3480      
                                                                 
 dense_2 (Dense)             (None, 20)                820       
                                                                 
 dense_3 (Dense)             (None, 1)                 21        
                                                                 
Total params: 11,717
Trainable params: 11,717
Non-trainable params: 0
_________________________________________________________________


In [24]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [25]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100)

Epoch 1/100
1881/1881 [==============================] - 4s 2ms/step - loss: 28.7089 - accuracy: 0.6110
Epoch 2/100
1881/1881 [==============================] - 3s 2ms/step - loss: 14.0583 - accuracy: 0.6236
Epoch 3/100
1881/1881 [==============================] - 3s 2ms/step - loss: 6.9620 - accuracy: 0.6345
Epoch 4/100
1881/1881 [==============================] - 3s 2ms/step - loss: 3.7231 - accuracy: 0.6474
Epoch 5/100
1881/1881 [==============================] - 3s 2ms/step - loss: 2.1080 - accuracy: 0.6702
Epoch 6/100
1881/1881 [==============================] - 3s 2ms/step - loss: 1.1575 - accuracy: 0.6971
Epoch 7/100
1881/1881 [==============================] - 3s 2ms/step - loss: 0.7910 - accuracy: 0.7208
Epoch 8/100
1881/1881 [==============================] - 3s 2ms/step - loss: 0.7120 - accuracy: 0.7258
Epoch 9/100
1881/1881 [==============================] - 3s 2ms/step - loss: 0.6281 - accuracy: 0.7362
Epoch 10/100
1881/1881 [==============================] - 3s 2ms/step -

In [26]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

627/627 - 1s - loss: 0.6376 - accuracy: 0.6881 - 838ms/epoch - 1ms/step
Loss: 0.6375783681869507, Accuracy: 0.688054621219635
